# Factor 

In [205]:
from pathlib import Path
import numpy as np
import pandas as pd
import quantstats as qs
import statsmodels.api as sm
from linearmodels.asset_pricing import LinearFactorModel
from statsmodels.regression.rolling import RollingOLS

%config InlineBackend.figure_format = "retina"

In [206]:
def cache(func):
    def wrapped_func(*args, **kwargs):
        table_name = func.__name__.split("_")[-1]
        parquet_path = f"./data/{table_name}.parquet"
        try:
            table = pd.read_parquet(parquet_path)
        except FileNotFoundError:
            table = func(*args, **kwargs)
            table.to_parquet(parquet_path)
        return table

    return wrapped_func

def query(sql_stmt, params, wrds_username):
    import wrds

    with wrds.Connection(wrds_username=wrds_username) as db:
        data = db.raw_sql(sql_stmt, date_cols=["date"], params=params)
    return data

def get_crsp(permnos, wrds_username):
    sql_crsp = """
    SELECT
        date,
        permno,
        openprc AS open,
        askhi AS high,
        bidlo AS low,
        prc AS close,
        vol AS volume,
        ret,
        shrout
    FROM
        crsp.dsf
    WHERE
        permno IN %(permnos)s
        AND date >= '1994-01-01'
        AND date <= '2019-12-31'
    ORDER BY
        date, permno;
    """
    params = {"permnos": permnos}
    crsp = query(sql_crsp, params, wrds_username)
    # Fill missing close prices of permno 80539
    crsp.loc[crsp.permno == 80539, "close"] = crsp.loc[crsp.permno == 80539, "close"].fillna(method="ffill")
    # Fill other missing values
    crsp = crsp.fillna({"open": crsp.close,
        "high": crsp.close,
        "low": crsp.close,
        "volume": 0,
        "ret": 0})
    # Calculate market capitalization
    crsp["cap"] = crsp.close * crsp.shrout
    # Shift market capitalization to avoid look ahead bias
    crsp["cap"] = crsp.groupby("permno").cap.shift(1)
    # Calculate market capiticalization weight
    crsp["w_cap"] = crsp.groupby("date").cap.apply(lambda x: x / x.sum())
    # Convert certain data types to int64
    crsp = crsp.astype({"permno":"int",
                        "shrout": "int",
                        "volume":"int"})
    return crsp

def get_fama_french():
    """
    Get 3-factor and momentum data from Ken French data library.
    """
    import pandas_datareader as web

    # Transfrom from percentage to nominal value
    factor = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench", start="1994-01-01", end="2019-12-31")[0]/100
    momentum = web.DataReader("F-F_Momentum_Factor_daily", "famafrench", start="1994-01-01", end="2019-12-31")[0]/100
    # Merge into 4 factor model
    fama_french = pd.concat([factor, momentum], axis=1)
    fama_french = fama_french.rename(columns=lambda x: x.lower().strip().replace("-", "")).rename_axis(index=str.lower)
    return fama_french

In [207]:
path = Path("./data")
permno_path = path/"permno_selection.csv"
permnos = pd.read_csv(permno_path, dtype={"permno":"str"}).squeeze().pipe(tuple)
crsp = get_crsp(permnos, "iewaij")
fama_french = get_fama_french()

Loading library list...
Done


## Estimate Factor Exposure

In [208]:
portfolio = crsp.loc[crsp.date >= "1994-01-01", ["date", "permno", "ret"]].pivot(
    index="date", columns="permno", values="ret"
)
portfolio = portfolio.sub(fama_french.loc[portfolio.index, "rf"], axis=0)
factor = fama_french.loc[portfolio.index, ["mktrf", "smb", "hml", "mom"]].assign(alpha=1)
betas = []
for permno in portfolio:
    ret = portfolio[permno]
    res = RollingOLS(endog=ret, exog=factor, window=756).fit(params_only=True)
    params = res.params
    params["resid"] = ret - factor.mul(params).sum(1)
    params["permno"] = permno 
    betas.append(params)
beta = pd.concat(betas).reset_index().dropna()

## Estimate Price Trend

In [209]:
ret = crsp.pivot(index="date", columns="permno", values="ret")
logret = np.log(ret+1)
# 1-month cumulative return
mom_1m = logret.rolling(21).sum()
# 11-month cumulative returns ending 1-month before
mom_12m = logret.shift(21).rolling(11*21).sum()
# Cumulative return from months t-6 to t-1 minus months t-12 to t-7.
mom_6m = logret.shift(21).rolling(5*21).sum()
mom_12m_6m = logret.shift(6*21).rolling(5*21).sum()
chmom = mom_6m - mom_12m_6m
# Max daily returns from calendar month t-1
maxret = logret.rolling(21).max()
# Cumulative returns months t-36 to t-13
mom_36m = logret.shift(12*21).rolling(24*21).sum()

## Estimate Liquidity

In [210]:
vol = crsp.pivot(index="date", columns="permno", values="volume")
shrout = crsp.pivot(index="date", columns="permno", values="shrout")
close = crsp.pivot(index="date", columns="permno", values="close") 
# Average monthly trading volume for most recent three months divided by number of shares
turn = vol.shift(21).mean().div(shrout)
# Monthly std dev of daily share turnover
turn_std = vol.div(shrout).rolling(21).std()
# Natural log of market cap
logcap = np.log(close) + np.log(shrout)
# Natural log of trading volume times price per share from month t-2
dv = vol.mul(close)
dolvol = np.log(dv.shift(21).rolling(21).mean())
# Average of daily (absolute return / dollar volume)
ill = ret.abs().div(dv)

/opt/homebrew/Caskroom/mambaforge/base/envs/quant/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


## Estimate Risk

In [211]:
# Standard dev of daily returns from month t-1
retvol = ret.rolling(21).std()
# Market beta
mktrf = beta.pivot(index="date", columns="permno", values="mktrf")
# Market beta squared
mktrf_sq = mktrf.pow(2)
# Idiosyncratic return volatility
idovol = beta.pivot(index="date", columns="permno", values="resid").rolling(756).std()

## Prepare Data

In [212]:
def combine_factor(**kwargs):
    factor = pd.DataFrame(
        {
            name: factor.resample("W-FRI").last().stack()
            for name, factor in kwargs.items()
        }
    )
    factor = factor.groupby("date").transform(lambda x: x.fillna(x.mean()))
    return factor


In [213]:
# Factor
factor = combine_factor(
    mom_1m=mom_1m,
    mom_12m=mom_12m,
    chmom=chmom,
    maxret=maxret,
    mom_36m=mom_36m,
    turn=turn,
    turn_std=turn_std,
    logcap=logcap,
    dolvol=dolvol,
    ill=ill,
    retvol=retvol,
    mktrf=mktrf,
    mktrf_sq=mktrf_sq,
    idovol=idovol,
)

In [214]:
# Forward 1 week return
ret_5d = np.exp(logret.resample("W-FRI").sum().stack().shift(-1)) - 1

In [215]:
X = factor["2000-01-01":"2012-12-31"] 
y = ret_5d["2000-01-01":"2012-12-31"]
X_train = factor["2000-01-01":"2007-12-31"]
X_valid = factor["2008-01-01":"2012-12-31"] 
y_train = ret_5d["2000-01-01":"2007-12-31"]
y_valid = ret_5d["2008-01-01":"2012-12-31"]

## Prediction

### OLS

In [216]:
# In-sample OLS
sm.OLS(endog=y, exog=X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.017
Model:                            OLS   Adj. R-squared (uncentered):              0.016
Method:                 Least Squares   F-statistic:                              41.06
Date:                Mon, 28 Mar 2022   Prob (F-statistic):                   1.17e-112
Time:                        18:11:50   Log-Likelihood:                          48474.
No. Observations:               33900   AIC:                                 -9.692e+04
Df Residuals:                   33886   BIC:                                 -9.680e+04
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mom_1m         0.0685      0.003     21.154      0.000       0.062       0.075
mom_12m       -0.0007      0.001     -0.745      0.456      -0.002       0.001
chmom          0.0016      0.001      1.896      0.058    -5.4e-05       0.003
maxret        -0.1388      0.023     -5.940      0.000      -0.185      -0.093
mom_36m       -0.0004      0.001     -0.683      0.495      -0.002       0.001
turn        6.209e-06   3.11e-05      0.200      0.842   -5.47e-05    6.71e-05
turn_std    3.991e-05   6.45e-05      0.618      0.536   -8.66e-05       0.000
logcap         0.0004      0.001      0.725      0.469      -0.001       0.002
dolvol        -0.0004      0.001     -0.745      0.456      -0.001       0.001
ill        -1.454e+05   1.54e+05     -0.945      0.345   -4.47e+05    1.56e+05
retvol         0.3797      0.059      6.482      0.000       0.265       0.495
mktrf          0.0017      0.003      0.684      0.494      -0.003       0.007
mktrf_sq      -0.0011      0.001     -0.864      0.388      -0.003       0.001
idovol         0.0003      0.049      0.005      0.996      -0.095       0.095
==============================================================================
Omnibus:                    10594.457   Durbin-Watson:                   1.640
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           351175.828
Skew:                           0.861   Prob(JB):                         0.00
Kurtosis:                      18.673   Cond. No.                     1.36e+10
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 1.42e-13. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""